# Training an RMN on all session documents

In [1]:
import os
import sys
import pandas as pd

In [2]:
sys.path.append("/home/rocassius/w266_final/scripts/assembly")
sys.path.append("/home/rocassius/w266_final/scripts/modeling")

In [3]:
from document import load_documents
from constant import DOC_ALL_PATH

In [4]:
from helper import load_pickled_object
from rmn import RMN
from rmn_data_generator import RMN_DataGenerator

In [5]:
# load embedding tools
local_tools_path = '/home/rocassius/gen-data/tools'
tokenizer_dict = load_pickled_object(os.path.join(local_tools_path, "global_tokenizer_dict"))
metadata_dict = load_pickled_object(os.path.join(local_tools_path, "global_metadata_dict"))
embedding_matrix = load_pickled_object(os.path.join(local_tools_path, "global_embedding_matrix_50d"))

In [6]:
# load documents
docs_df = load_documents([111], DOC_ALL_PATH)

In [7]:
# correct types
docs_df['speakerid'] = docs_df['speakerid'].astype(str)
docs_df['session'] = docs_df['session'].astype(str)

In [8]:
d = docs_df.sample(500)

In [43]:
#==================#
#=*= RMN Module =*=#
#==================#

# RMN Class for training Relationship Modeling Networks 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import numpy as np

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Embedding, Dense, Lambda, Input, Masking, Reshape, Concatenate
from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.regularizers import Regularizer

from rmn_data_generator import RMN_DataGenerator
from helper import pickle_object, load_pickled_object
from vector_math import find_nn_cos

# constants
MAX_SPAN_LENGTH = 50
NUM_TOPICS = 20
LAMBDA = 5.0

# hyperparameters
OPTIMIZER = 'adam'
BATCH_SIZE = 50
EPOCHS = 5

# saving tags
RMN_TAG = "rmn_%s"
MODEL = "model.h5"
ATTR = "attributes"

# attribute keys
N_TOP_KEY = 'num_topics'
LAMB_KEY  = 'lambda'
EMBED_KEY = 'emedding_matrix'
TOKEN_KEY = 'tokenizer_dict'
META_KEY  = 'metadata_dict'


class RMN(object):
    """
    Class for constructing a Relationship Modeling Network
    """
    
    def __init__(self):
        
        # model parameters
        self.num_topics = NUM_TOPICS
        self.lamb = LAMBDA
        
        # model attrbiutes
        self.embedding_matrix = None
        self.tokenizer_dict = None
        self.metadata_dict = None
        
        # models 
        self.model = None
        self.topic_model = None
        
    
    @property
    def embedding_dim(self):
        return self.embedding_matrix.shape[1]
    
    
    def model_loss(self):
        """Hinge loss function.
        """
        def custom_loss(y_true, y_pred):
            
            # hinge_loss
            y_true_normalized = K.l2_normalize(y_true, axis=-1)
            y_pred_normalized = K.l2_normalize(y_pred, axis=-1)
            dot_product = K.sum(y_true_normalized * y_pred_normalized, axis=-1)
            hinge_loss = K.mean(K.maximum(0., 1. - dot_product))

            return hinge_loss 

        return custom_loss
    
    
    def build_model(self):
        """Connstruct the RMN model architecture
        """
        # Span Input
        span_input = Input(shape=(self.tokenizer_dict['max_span_length'],), 
                           name='Span.Input')
        span_embedding = Embedding(input_dim=len(self.tokenizer_dict['word_index']) + 1, 
                                   output_dim=self.embedding_dim, 
                                   weights=[self.embedding_matrix],
                                   input_length=self.tokenizer_dict['max_span_length'],
                                   trainable=False)(span_input)
        
        # Take elementwise average over vectors
        span_avg = Lambda(lambda x: K.mean(x, axis=1), name = "Span.Avg.Layer")(span_embedding)

        input_layers = [span_input]
        embedding_layers = [span_avg]
        
        for col in self.metadata_dict.keys():
            
            input_layer = Input(shape=(1,), name= col + '.Input')
            
            # embedding layer for col
            embedding_init = Embedding(
                input_dim = self.metadata_dict[col]['input_dim'] + 1, 
                output_dim = self.embedding_dim,
                input_length = 1)(input_layer)
            
            # reshape
            embedding_layer = Reshape((self.embedding_dim, ), name=col + '.Embed.Layer')(embedding_init)
            
            input_layers.append(input_layer)
            embedding_layers.append(embedding_layer)

        # concat speaker metadata embeddings
        _ht = Concatenate(axis=1, name = 'Concat.Layer')(embedding_layers)

        # dense layer
        ht = Dense(units = self.embedding_dim, 
                   input_shape = (_ht.shape[1], ), 
                   activation = "relu", name = "Wh")(_ht)

        # dense layer with softmax activation, (where previous states will eventually be inserted) 
        dt = Dense(units = self.num_topics, 
                   input_shape = (self.embedding_dim, ), 
                   activation = "softmax", name = "Wd")(ht)

        # reconstruction layer
        rt = Dense(units = self.embedding_dim,
                   input_shape = (self.num_topics, ),
                   activation = "linear",
                   kernel_regularizer = Orthogonality(self.lamb),
                   name = "R")(dt)

        # compile
        model = tf.keras.Model(inputs=input_layers, outputs=rt)
        model.compile(optimizer = OPTIMIZER, loss = self.model_loss())
        self.model = model
        
        # build associated topic model
        self.build_topic_model()
        
        
    def build_topic_model(self, topic_layer = "Wd"):
        """Contruct model whose output is the topic distribution layer
        """
        topic_model = tf.keras.Model(
            inputs = self.model.input,
            outputs = self.model.get_layer(topic_layer).output)
        
        self.topic_model = topic_model
          
    
    def prep_spans(self, documents):
        """Returns the lists of word ids associated with the text
        """
        return self.tokenizer_dict['tokenize_pad'](documents)
    
    
    def prep_metadata(self, df):
        """Preps metadata for training or prediction
        """
        metadata_x = [np.array(self.metadata_dict[col]['tokenize'](df[col]))
                      for col in self.metadata_dict.keys()]

        return metadata_x
        
    
    def prep_inputs(self, df, for_training=False):
        """Preps metadata and spans for training or prediction
        """
        spans_y = self.prep_spans(df['document'])
        metadata_x = self.prep_metadata(df)
        X = [spans_y] + metadata_x
        
        if for_training:
            y = self.embedding_matrix[spans_y].mean(axis=1)
            return X, y
        else:
            return X
    
    
    
# Orthogonality Regularizer #

class Orthogonality(Regularizer):
    """Regularizer for discouraging non-orthogonal components.
    
    # Arguments
        lamb: Float; regularization penalty weight
    """

    def __init__(self, lamb = 1.):
        self.lamb = lamb

    def __call__(self, R):
        RRT = K.dot(R, K.transpose(R))
        I = K.eye(int(RRT.shape[0]))
        penalty = self.lamb * K.sqrt(K.sum(K.square(RRT - I)))
        
        return penalty
    
    
# Topic Parsimony Regularizer #

In [44]:
rmn = RMN()
rmn.embedding_matrix = embedding_matrix
rmn.tokenizer_dict = tokenizer_dict
rmn.metadata_dict = metadata_dict
rmn.num_topics = 100
rmn.lamb = 1.0
rmn.build_model()

In [46]:
#rmn.model.summary()

In [48]:
X, y = rmn.prep_inputs(docs_df, for_training=True)

In [35]:
X[0].shape

(142590, 80)

In [49]:
y.shape

(142590, 50)

In [51]:
rmn.model.fit(X, y, epochs=2, batch_size=500)

Train on 142590 samples
Epoch 1/2
142590/142590 [==============================] - 3s 23us/sample - loss: 7.4029
Epoch 2/2
142590/142590 [==============================] - 3s 23us/sample - loss: 7.0838


In [121]:
from tensorflow.keras.layers import Embedding, Dense, Lambda, Input, Masking, Reshape, Concatenate

In [122]:
span_input = Input(shape=(75,), name='Span.Input')
    
span_embedding = Embedding(input_dim=100 + 1, 
                           output_dim=50, 
                           input_length=75)(span_input)


In [123]:
span_embedding.shape

TensorShape([Dimension(None), Dimension(75), Dimension(50)])

In [124]:
span_embedding

<tf.Tensor 'embedding_65/embedding_lookup/Identity_1:0' shape=(?, 75, 50) dtype=float32>

In [126]:
lstm = LSTM(4, input_shape=(75,50,))(span_embedding)

AttributeError: 'Node' object has no attribute 'output_masks'

In [114]:
from keras.models import Sequential
from keras.layers import SpatialDropout1D
import keras
model_lstm = Sequential()
model_lstm.add(keras.layers.Embedding(input_dim = 60, output_dim = 256, input_length = 70))
model_lstm.add(keras.layers.SpatialDropout1D(0.3))
model_lstm.add(keras.layers.LSTM(256, dropout = 0.3, recurrent_dropout = 0.3))
model_lstm.add(keras.layers.Dense(256, activation = 'relu'))
model_lstm.add(keras.layers.Dropout(0.3))
model_lstm.add(keras.layers.Dense(5, activation = 'softmax'))

In [115]:
model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [116]:
model_lstm.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 70, 256)           15360     
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 70, 256)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 607,749
Trainable params: 607,749
Non-trainable params: 0
_______________________________________________